**ATENCIÓN:** Executar la siguiente celda solamente si se esta corriendo este notebook desde Google Colab.

In [ ]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'dlvis/assignments/assignment2/'
FOLDERNAME = None
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# this downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
!bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME

# Computer Vision MNIST

En este notebook se abordaran algunas de las funcionalidades básicas de Keras y Tensorflow, necesarias para cargar un dataset en memoria, definir la arquitectura de una Red Neuronal y posteriormente entrenarla para luego utiizarla en la inferencia de algunas imágenes.

A su vez, la redes neuronales utilizan vectores de numpy por lo que veremos como se representa una imagen como vector de numpy y que propiedades tiene.

# Tabla de Contenidos

Este notebook contiene 5 partes. Vamos a repasar algunas funcionalidades básicas de TensorFlow y Keras entrenando un modelo para reconocer dígitos escritos a mano.

1. Parte I, Preparación: cargar dataset MNIST.
2. Parte II, Entendimiento: visualizar datos y obtener algunas estadísticas.
3. Parte III, Entrenamiento: definición y entrenamiento de modelo.
4. Parte IV, Evaluación: evaluar el modelo y predecir algunos casos.
5. Pate V, Experimentación: Algunas opciones como Dropout y Optimizadores

# Parte I: Preparación

Primero, vamos a descargar el dataset MNIST. Esto puede tomar algunos minutos para descargar los datos por primer vez, pero luego de que los archivos son descargados y cacheados en disco la carga debería ser más rápida.

Si bien es común trabajar con un dataset particular, por simplicidad vamos a trabajar un poco con el módulo ```tf.keras.datasets``` que ya provee utilidades para cargar y trabajar con cualquiera de los siguiente datasets clásicos:

* MNIST digits classification
* CIFAR10 small images classification
* CIFAR100 small images classification
* IMDB movie review sentiment classification
* Reuters newswire classification
* Fashion MNIST
* Boston Housing price reression

En este taller vamos a utilizar los datasts MNIST y CIFAR100. Por más información acerca de etos datasets leer el siguiente [link](https://keras.io/api/datasets/).

Por otro lado, TensorFlow cuenta con el módulo `tf.data` para cargar un dataset cualquiera a partir de archivos CSV o un directorio de imágenes. Por más información referise al siguiente [link](https://www.tensorflow.org/guide/data).

In [ ]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def load_mnist(num_training=58000, num_validation=2000, num_test=10000):
    """
    Fetch the MNIST dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier.
    """
    # Load the raw MNIST dataset and use appropriate data types and shapes
    mnist = tf.keras.datasets.mnist.load_data()
    (X_train, y_train), (X_test, y_test) = mnist
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_mnist()

Notar los siguientes aspectos de la función `load_mnist()`:

* Separa una proporción `num_validation` de los datos de entrenamiento para utilizar como datos de validación
* Tanto los datos de Train, Val como Test son normalizados utilizando la media (`mean_pixel`) y la desviación estandar (`std_pixel`) de los valores de los pixeles en la imagen. Se sabe que esto ayuda a las Redes Neuronales a obtener mejores resultados.

In [ ]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [ ]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

Opcionalmente puedes **usasr GPU seteando la siguiente flag en True** en la siguiente celda.

## Usuarios Colab

Si estas usando Colab, seguramente necesites manualmente cambiar a un entorno GPU. La forma de hacer esto es clickeando la opcion `Runtime -> Change runtime type` y seleccionar la opción `GPU` dentro de `Hardware Accelerator`. Notar que debes correr de nuevo todas las celdas de arriba ya que al cambiar de entorno de ejecución el kernel del notebook se reinicia perdiendo el estado actual.

In [ ]:
# Set up some global variables
USE_GPU = True

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

# Parte II: Entendimiento de los datos

En segundo lugar echemos un vistaso a las imágnes con las que vamos a trabajar el resto del notebook para entender cuantos datos tenemos para trabajar y preveer cualquier potencial problema antes de entrenar cualquier modelo.

En un dataset compuesto por imágenes y en particular en la tarea de clasificación usualmente se chequean los siguientes puntos antes de empezar a entrenar un modelo:

* ¿Cuántas imágenes tenemos para Train y cuántas para Test?
* ¿Cuántas imágenes tenemos por cada clase, está balanceado?
* ¿Qué resolución tiene cada imagen? ¿Son todas iguales o hay que aplicar algún re-size?
* ¿Cuantos canales tiene la imágen (blanco y negro o a color)?

Además, es una buena práctica visualizar algunas imágenes por cada clase, para comprender bien como son las imágenes con las que estaremos trabajando.

Empecemos entonces a contestar estas preguntas!!

**¿Cuántas imágenes tenemos?**

Veamos algunas estadísticas generales del dataset.

In [ ]:
# Print some basic stats about the dataset
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

**¿Cuántas imágenes tenemos por cada clase?**

Como se puede ver en las celdas a continuación, si bien el dataset no contiene exactamente la misma cantidad de ejemplos para cada clase, se puede decir que está balanceado, contando con al menos 5200 ejemplos para cada clase y a lo sumo 6500. 

Se puede decir entonces que al menos en cantidad, es un dataset suficiente para comenzar a trabajar.

In [ ]:
# Compute unique values on y_train
unique, counts = np.unique(y_train, return_counts=True)

print(dict(zip(unique, counts)))

In [ ]:
# Display as bar plot

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(unique, counts)

plt.xticks(unique)  # To display each value of
plt.show()

**¿Que sucede con el dataset de test?** Como se puede ver en la figura debajo, también se encuentra balanceado, con cerca de 1000 ejemplos por clase.

In [ ]:
############################################################################
# TODO: Calcular cantidad de ejemplos por clase en y_test y graficar       #
############################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
############################################################################
#                            END OF YOUR CODE                              #
############################################################################

**¿Qué resolución tiene cada imagen?**

Como se puede ver debajo, el dataset de entrenamiento se compone de 58.000 imágnes de 28x28 pixeles. En este caso entonces no tenemos nada de que preocuparnos, todas las imágenes tienen la misma resolución. En otros datasets con los que aveces trabajamos esto no sucede, por lo que es necesario aplicar un paso de re-size en todas las imágenes antes de continuar trabajando, para algunas arquitecturas de redes.

In [ ]:
X_train.shape

**Entendiendo las imágenes**

Como se puede ver a continuación, cada imágen contiene un solo dígito en blanco y negro. A este tipo de imágenes se las conoce como máscaras binarias también, ya que contiene en color blanco solo los pixeles que pertenecen al objeto en cuestión (en este caso un dígito) y en negro aquellos pixeles que pertenecen al fondo de la imágen.

En deep learning se suelen trabajar con imágenes a color y en blanco y negro como es este caso. Cuando las imágenes son en blanco y negro comoe este caso, se dice que la imagen tiene un solo canal (1 o cero). En caso que la imagen sea a color se suele trabajar con el sistema RGB, por lo que se dice que la imagen tiene 3 canales. En este caso, notar que la matriz que representa la imagen ya no sería de 28x28 pixeles.

_¿Qué dimensión tendría?
HINT: Pensar en el color de cada pixel representado por el sistema RGB._

In [ ]:
def display_mnist_images(X, y, num=25, num_row=5, num_col=5):
    """
    Display on a grid using matplotlib train images and their corresponding
    label.
    """
    
    # Get images to display
    images = X[:num]
    labels = y[:num]

    # plot images
    fig, axes = plt.subplots(num_row, num_col, figsize=(1.5*num_col,2*num_row))
    for i in range(num):
        ax = axes[i//num_col, i%num_col]
        ax.imshow(images[i], cmap='gray')
        ax.set_title('Label: {}'.format(labels[i]))
    plt.tight_layout()
    plt.show()

display_mnist_images(X_train, y_train)

# Parte III: Entrenamiento

Ahora que tenemos un mejor entendimiento del problema y las imágenes con las que contamos, pasemos a definir y entrenar el modelo que clasifique las imágenes.

Para esto vamos a utilizar la clase `tf.keras.Sequential` disponible en la libreria Keras, que permite definir una red neuronal con un número arbitrario de `layers`. Por más información acerca de esta clase leer [tf.keras.Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) la documentación.

Para entrar en calentamiento lo que vamos a hacer, es definir una sencilla Red Neuronal con 1 hidden layer y Relu como función de activación.

Resumen:
* Input Layer: Imágenes de 28x28 como vector usando la clase [tf.keras.layers.Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten) que toma una matrix de cualquier tamaño y la convierte en un vector.
* Hidden Layer: Capa oculta de 128 neuronas con Relu. Leer más sobre [tf.keras.layers.Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense).
* Output Layer: Nuevamente usamos `tf.keras.layers.Dense` pero con largo 10 (por 10 diferentes clases).

In [ ]:
# Define model architecture using keras.Sequential API

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),  # Flatten of img input size
    tf.keras.layers.Dense(128, activation='relu'),  # Dense of arbitrary hidden neurons size
    tf.keras.layers.Dense(10)  # Dense of num_classes length
])

Una vez definida la arquitectura del modelo es necesario compilarlo, indicando entre otras clases optimizador a utilizar para el entrenamiento, que función de costo vamos a utilizar y la métrica de evaluación. Leer más acerca de [tf.keras.compile](https://www.tensorflow.org/api_docs/python/tf/keras/Model). 

En este caso vamos a utilizar:
* Optimizador: SGD con momentum. Leer más en [keras optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers).
* `SparseCategoricalCrossentropy`: Entropía cruzada comunmente utilizada como función de costo en problemas de clasificación. Leer más en [SparseCategoricalCrossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy)


In [ ]:
# Compile model
model.compile(optimizer='SGD',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Ahora solo resta entrenar el modelo tantas `epochs` como creamos conveniente.

In [ ]:
# Train model
model.fit(X_train, y_train, epochs=10)

# Parte IV: Evaluación

Como podemos ver del loss log durante el entrenamiento, con esta simple arquitectura se puede sobre-ajustar los datos de entrenamiento, logrando un `loss < 0.03` y una `accuracy` superior al `0.99`.

Sin embargo, no siempre los resultados en la fase de Train se asemejan a los resultados sobre datos reales o en este caso, sobre los datos que separamos para Test. Por ello, veamos como nos va con la `accuracy` sobre los datos de Test.

Para ello vamos a utilizar la función `evaluate` de la propia clase `keras.Model`.

In [ ]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

### Test con imagen arbitraria

Veamos como interactuar con el modelo para predecir el resultado de una imagen arbitraria en el dataset de test.

In [ ]:
# Change num_sample to test with a different image
num_sample = 50
image = X_test[num_sample]
label = y_test[num_sample]

plt.imshow(image, cmap='gray')
plt.show()

In [ ]:
# Next cell call model using predict method. Recall in the fact model expects to recive an array of 
# examples instead of a single example. For this reason is neccesary to expand dims of image to an array with a
# single image.
# For the same reason predictions is a list of predictions
predictions = model.predict(np.expand_dims(image, axis=0)) 

prediction = np.argmax(predictions[0])  # prediction is an array of 10 probabilities, the highest is the predicted class

print(f"Prediction: {prediction} and expected is: {label}")

# Parte IV: Experimentación

Ahora que tenemos claro como definir una arquitectura de Red Neuronal con Keras y entranarla, te proponemos que  pruebes algunas varianetes del modelo original para ver si es posible mejorar la `accuracy` sobre los datos de Test y llegar a un valor similar al `0.99` obtenido sobre los datos de Train.

A modo de guia te sugerimos algunas de las siguientes variantes para probar:

* Cambios en arquitectura: Cambiar el número de neuronas en la capa oculta (Dense) para incrementar / decrementar la capacidad del modelo.
* Optimizadores: En el modelo original se utiliza SGD (Stochastic gradient descent), probar con otros optimizadores como [RMSprop o Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers).
* Dropout: 

In [ ]:
############################################################################
# TODO: Calcular cantidad de ejemplos por clase en y_test y graficar       #
############################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

model = tf.keras.Sequential([
    ...  # TODO: Edit this
])

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
############################################################################
#                            END OF YOUR CODE                              #
############################################################################

In [ ]:
############################################################################
# TODO: Calcular cantidad de ejemplos por clase en y_test y graficar       #
############################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

model.compile(optimizer=pass,  # TODO: Select an optimizer
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
############################################################################
#                            END OF YOUR CODE                              #
############################################################################

In [ ]:
############################################################################
# TODO: Calcular cantidad de ejemplos por clase en y_test y graficar       #
############################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

model.fit(X_train, y_train, epochs=pass)  # TODO: select num of epochs

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
############################################################################
#                            END OF YOUR CODE                              #
############################################################################

In [ ]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

**Me siento curioso**:

Si te sientes curioso acerca de como ir más allá del 99% de accuracy sobre MNIST y no lograste dar con la respuesta te recomiendo continuar tu investigación [aquí](https://towardsdatascience.com/going-beyond-99-mnist-handwritten-digits-recognition-cfff96337392).